In [1]:
import sys
sys.path.append('../../source_code/')

from scGCO_source import *

#from scGCO import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2,venn3
%matplotlib inline

import time
%load_ext memory_profiler

In [2]:
import warnings
warnings.filterwarnings('ignore')
import gc 

In [3]:
def clear():

    for key, value in locals().items():
        del locals()[key]
    gc.collect()

### propressing input data of scGCO

In [9]:
ff='../../../data/Simulation_data/1M_cells_100genes_counts_0.csv'
%memit data0 = pd.read_csv(ff, sep=',', index_col = 0,iterator=True)
i=1000000
data=data0.get_chunk(i)

peak memory: 217.68 MiB, increment: 9.46 MiB


In [ ]:
data= data.loc[:,(data != 0).astype(int).sum(axis=0) >= 10]
data_norm= normalize_count_cellranger(data)


In [ ]:

t0=time.time()
gmmDict= multiGMM(data_norm)
t1=time.time()
print(t1-t0)

f=open("../../supple_data/1M_cells_100genes_gmmDict.plk",'wb')
pickle.dump(gmmDict)
print(time.time() - t1)

In [5]:
nc=[]
tt =[]
import time
n_cell=[100,250,500,1000,1500,5000,10000,15000,50000,100000,500000,1000000]
for i in n_cell:
    ff='../../../Simulation_data/1M_cells_100genes_data_norm_0.csv'
    data_norm0 = pd.read_csv(ff, sep=',', index_col = 0,iterator=True)
    data_norm=data_norm0.get_chunk(i)

    locs=Get_coord(data_norm)
    
    exp= data_norm.iloc[:,0]
    cellGraph= create_graph_with_weight(locs, exp)
 
    smooth_factor=10
    start = time.time()
    %memit result_df = identify_spatial_genes_optimize_gmm(locs,data_norm,cellGraph,gmmDict,smooth_factor)
    end = time.time()
    t=end-start
    
    nc.append(int(i))
    tt.append(t)
    print(i, ' time: ', t)
    
   
    clear()

100%|██████████| 8/8 [00:00<00:00, 17.81it/s]


peak memory: 229.85 MiB, increment: 1.24 MiB
100  time:  0.7302708625793457


100%|██████████| 8/8 [00:00<00:00,  9.12it/s]


peak memory: 231.94 MiB, increment: 1.31 MiB
250  time:  1.160381555557251


100%|██████████| 8/8 [00:01<00:00,  5.44it/s]


...

peak memory: 987.55 MiB, increment: 411.15 MiB
100000  time:  438.1465427875519


100%|██████████| 8/8 [40:14<00:00, 301.75s/it]   


peak memory: 3737.52 MiB, increment: 2563.14 MiB
500000  time:  2432.1537821292877


100%|██████████| 8/8 [1:27:14<00:00, 654.27s/it]   


peak memory: 7811.83 MiB, increment: 5114.87 MiB
1000000  time:  5271.786563158035


In [6]:
tt

[0.7302708625793457,
 1.160381555557251,
 1.7411181926727295,
 3.2735602855682373,
 4.78583836555481,
 15.837266683578491,
 32.93152594566345,
 68.35161447525024,
 214.4603304862976,
 438.1465427875519,
 2432.1537821292877,
 5271.786563158035]

In [8]:
memory= np.array([229.85, 231.94, 234.95, 240.35, 248.00, 268.50, 307.96, 347.89, 600.31, 987.55, 3737.52, 7811.83])

In [11]:
df=pd.DataFrame({'n.cells':nc,'n.genes':100,'time(s)':tt,'memory(MiB)':memory})

In [12]:
df

,n.cells,n.genes,time(s),memory(MiB)
0,100,100,0.730271,229.85
1,250,100,1.160382,231.94
2,500,100,1.741118,234.95
3,1000,100,3.273560,240.35
4,1500,100,4.785838,248.00
5,5000,100,15.837267,268.50
6,10000,100,32.931526,307.96
7,15000,100,68.351614,347.89
8,50000,100,214.460330,600.31
9,100000,100,438.146543,987.55
